<center>
<img src="https://www.iybssd2022.org/wp-content/uploads/ASAQ.jpg" width="150"/> 
</center>

        
<center>
<h1><font color= "blue" size="+2">ASAQ Python Data Analysis Courses</font></h1>
</center>

---

<center><h1><font color="blue" size="+2">Analysis with GeoPandas</font></h1></center>

## <font color="red">Objectives</font>

We want to:

- Provide an overview of GeoPandas and its main data structures.
- Learn how to create and manipulate a GeoDataFrame.
- Plot data along a path.

## <font color="red">Required modules/packages</font>

- __Matplotlib__: for basic plots.
- __Pandas__: Manipulation and exploratory data analysis of tabular data.
- __Shapely__: For manipulation and analysis of planar geometric objects
- __GeosPandas__: Combines the capabilities of Pandas and Shapely for geospatial operations.

### <font color="red">Uncomment and run the cell below only if in Google Colab</font>

In [ ]:
#!sudo apt-get update && apt-get install -y libspatialindex-dev
#!pip install rtree
#!pip install geopandas
#!pip install mapclassify

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
print(f"Pandas    version: {pd.__version__}")
print(f"GeoPandas version: {gpd.__version__}")

# What is GeoPandas

- A Python library that allows you to process shapefiles representing tabular data (like Pandas), where every row is associated with a geometry.
- Designed to primarily work with vector data.
- Provides access to many spatial functions for applying geometries, plotting maps, and geocoding. 
- Extends the capabilities of Pandas to enable spatial operations. 
- Includes new data types such as `GeoDataFrame` and `GeoSeries` which are subclasses of Pandas DataFrame and Series and enables efficient vector data processing in Python. 
- Is built on top of the following libraries that allow it to be spatially aware:
  - `Shapely`: geometric operations (i.e. buffer, intersections etc.)
  - `PyProj`: working with projections
  - `Fiona`: file input (reading) and output (writing).

![fig_frame](https://geopandas.org/en/stable/_images/dataframe.svg)
Image Source: [GeoPandas](https://geopandas.org/en/stable/getting_started/introduction.html)

# <font color="red"> Creating GeoDataFrame </font>

- We start with a Pandas DataFrame that has latitude and longitude coordinates as columns representing locations of cities.
- We perform transformations to create a GeoPandas GeoDataFrame that includes the "geometry" column (representing points).

In [ ]:
cities = ['Paris', 'New York', 'Mumbai', 'Tokyo', 
          'Moscow', 'Mexico City', 'Sao Paulo', 'Yaounde', 
          'Vancouver', 'Sydney', 'Harare']
countries = ['France', 'USA', 'India', 'Japan', 
             'Russia', 'Mexico', 'Brazil', 'Cameroon', 
             'Canada', 'Australia', 'Zimbabwe']
longitudes = [2.25, -73.92, 72.83, 139.69, 37.36, -99.13, 
              -46.63, 11.50, -123.08, 151.20, 31.0]
latitudes = [48.85, 40.69, 28.35, 35.68, 55.45, 19.43,
             -23.55, 3.84, 49.32, -33.87, -18.0]

cities_df = pd.DataFrame({
    'City': cities,
    'Country': countries,
    'Longitude': longitudes,
    'Latitude': latitudes
})

cities_df

#### We zip the `Latitude` and `Longitude` together to create a new column named `Coordinates`.

In [ ]:
cities_df["Coordinates"] = list(zip(cities_df.Longitude, 
                                    cities_df.Latitude))
cities_df

In [ ]:
type(cities_df.Coordinates[0])

- We turn the `Coordinates` tuple into a Shapely `Point` object.
    - Apply Shapely’s `Point` method to the `Coordinates` column.

In [ ]:
cities_df["Coordinates"] = cities_df["Coordinates"].apply(shpgeom.Point)
cities_df

In [ ]:
type(cities_df.Coordinates[0])

- Finally, we will convert our DataFrame into a GeoDataFrame by calling the `geopandas.DataFrame` method.
- GeoDataFrame is a data structure with the convenience of a normal DataFrame but also an understanding of how to plot maps.

>The most important property of a GeoDataFrame is that it always has one GeoSeries column that holds a special status. This GeoSeries is referred to as the GeoDataFrame’s “geometry”. When a spatial method is applied to a GeoDataFrame (or a spatial attribute like area is called), this commands will always act on the “geometry” column.

In [ ]:
cities_gdf = gpd.GeoDataFrame(cities_df, geometry="Coordinates")
cities_gdf.head()

In [ ]:
cities_gdf.plot();

## <font color="red">Data Access</font>

File name:

In [ ]:
file_name = "L2-01-08-2023evening.xlsx"
data_url = "https://github.com/JulesKouatchou/asaq_py/raw/main/sample_data/L2-01-08-2023evening.xlsx"
data_url = "/".join(["../sample_data", file_name])

### <font color="blue">Read the file</font>

In [ ]:
df = pd.read_excel(data_url, 
                   sheet_name="Feuil1",
                  parse_dates={'t': [0]}
                  )
df

We can remove the column _time_:

In [ ]:
df = df.drop(columns=['time'])

#### Make the time as the index of the DataFrame

In [ ]:
df.set_index('t', inplace=True)

## Create a GeoDataFrame

In [ ]:
df["Coordinates"] = list(zip(df.lng, df.lat))
df

In [ ]:
df["Coordinates"] = df["Coordinates"].apply(shpgeom.Point)
df

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry="Coordinates")
gdf.head()

In [ ]:
fig, axes = plt.subplots(figsize=(7, 112))

gdf.plot(ax=axes, color='red', markersize=1);

In [ ]:
fig, axes = plt.subplots(figsize=(7, 112))
gdf.plot(column='CO2', ax=axes, legend=True);

#### Resize the colorbar

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(7,12))

divider = make_axes_locatable(axes)
cax = divider.append_axes("right", size="5%", pad=0.1)

gdf.plot(column='CO2', ax=axes, legend=True, 
         cmap='jet', cax=cax);

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world.head(5)

In [ ]:
world.plot();

In [ ]:
africa = world[world['continent']=='Africa']

In [ ]:
africa.plot(cmap='CMRmap');

In [ ]:
world.boundary.plot();

In [ ]:
base = africa.boundary.plot(figsize=(9,12));
gdf.plot(ax=base, color='red')#, marker='o', color='red', markersize=5)

In [ ]:
sen_url = "https://data.humdata.org/dataset/bd9bc484-155d-41a3-87cf-064310a94492/resource/4ef61299-7edb-4529-aa09-76137c14962a/download/sen_admbnd_anat_20240520_ab_shp.zip"

In [ ]:
sen_gdf = gpd.read_file(sen_url)

In [ ]:
sen_gdf

In [ ]:
sen_gdf.boundary.plot();

In [ ]:
base = sen_gdf.boundary.plot(figsize=(9,12));
gdf.plot(ax=base, marker='o', color='red', markersize=1);